In [22]:
import numpy as np 
import os
import pandas as pd
import tensorflow as tf

PATH = 'd:/Github/meaning_map/'
# PATH = '/Users/macair/Dropbox/4. Project/paper/'
# PATH = 'D:/Dropbox/4. Project/paper/'

In [23]:
# df = pd.DataFrame()
# data = []
li = []

for i in range(1):
    data = pd.read_csv(PATH + 'dataset/scopus(' + str(i) + ').csv', error_bad_lines=False)
    # data = pd.read_csv(PATH + 'dataset/scopus(' + str(i) + ').csv')
    li.append(data)

    # for idx, val in df.iterrows():

    #     for vv in val['Abstract'].split('.'):
    #         data.append(vv)
    
df = pd.concat(li, axis=0, ignore_index=True)
print(df.shape) 

(1782, 31)


In [24]:
print(df[:3])

                                     Authors  \
0            Tyagi D., Soni V.K., Khare V.K.   
1  Dong L., Wang S., Liu Y., Jin Y., Liao X.   
2                     Rong C.C., Barnes P.N.   

                                               Title  Year  \
0  Evaluation of critical success factors for six...  2017   
1  Design of Superconducting Energy-storage Pulse...  2017   
2  Developmental Challenges of SMES Technology fo...  2017   

                                        Source title Volume Issue Art. No.  \
0  International Journal of Mechanical and Produc...      7     6      NaN   
1           Gaodianya Jishu/High Voltage Engineering     43    12      NaN   
2  IOP Conference Series: Materials Science and E...    279     1   012013   

  Page start Page end  Page count  ...      ISSN ISBN  CODEN PubMed ID  \
0        489      500         NaN  ...  22496890  NaN    NaN       NaN   
1       4006     4012         NaN  ...  10036520  NaN  GAJIE       NaN   
2        NaN      NaN  

In [25]:
headline = [] # 리스트 선언
headline.extend(list(df.Abstract.values)) # 헤드라인의 값들을 리스트로 저장
headline[:5] # 상위 5개만 출력

['This research paper is to evaluate and present the results from the online survey conducted in various Indian manufacturing small and medium enterprises (SMEs), mainly to focus critical success factors (CSFs) for implementing Six Sigma in Indian manufacturing small and medium enterprises and identified most crucial critical success factors for Six Sigma implementation in SMEs of manufacturing sector in India. The paper is based on survey questionnaire apposite for Indian manufacturing SMEs and the result investigation of the present study is based on factor analysis and descriptive statistics. The results are investigated by the factor analysis and reveal the impact of different CSFs on the Indian manufacturing SMEs. © TJPRC Pvt. Ltd.',
 'According to the application requirements of the railgun, we put forward a novel circuit topology of superconducting magnetic energy storage(SMES) pulsed power supply based on STRETCH meat grinder circuit to solve the problems of eliminating arc at 

In [26]:
print('총 샘플의 개수 : {}'.format(len(headline))) # 현재 샘플의 개수

총 샘플의 개수 : 1782


In [27]:
from string import punctuation

def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower() # 구두점 제거와 동시에 소문자화

# text = [repreprocessing(x) for x in headline.split('.')]

text = []
for head in headline:
#     text.append([repreprocessing(x) for x in h.split('.')])
    for h in head.split('.'):
        text.append(repreprocessing(h))


text[:5]

['this research paper is to evaluate and present the results from the online survey conducted in various indian manufacturing small and medium enterprises smes mainly to focus critical success factors csfs for implementing six sigma in indian manufacturing small and medium enterprises and identified most crucial critical success factors for six sigma implementation in smes of manufacturing sector in india',
 ' the paper is based on survey questionnaire apposite for indian manufacturing smes and the result investigation of the present study is based on factor analysis and descriptive statistics',
 ' the results are investigated by the factor analysis and reveal the impact of different csfs on the indian manufacturing smes',
 '  tjprc pvt',
 ' ltd']

In [28]:
print(len(text))

16922


In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 15178


In [30]:
sequences = list()

for line in text: # 14956 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[10, 19],
 [10, 19, 28],
 [10, 19, 28, 7],
 [10, 19, 28, 7, 4],
 [10, 19, 28, 7, 4, 712],
 [10, 19, 28, 7, 4, 712, 3],
 [10, 19, 28, 7, 4, 712, 3, 241],
 [10, 19, 28, 7, 4, 712, 3, 241, 1],
 [10, 19, 28, 7, 4, 712, 3, 241, 1, 36],
 [10, 19, 28, 7, 4, 712, 3, 241, 1, 36, 23],
 [10, 19, 28, 7, 4, 712, 3, 241, 1, 36, 23, 1]]

In [31]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : individual


In [32]:
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 243


In [33]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0 10 19]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0 

In [34]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [35]:
print(X[:3])

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0 10]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0 

In [36]:
print(y[:3]) # 레이블

[19 28  7]


In [37]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(y, num_classes=vocab_size)

In [45]:

# print(X[:3])
# print('------------')
# print(y[:3])
print(y[0])

for yy in y[:10]:

    if 1 in yy:
        # print(1)
        print(len(yy))
        print(yy)
        


[0. 0. 0. ... 0. 0. 0.]
15178
[0. 0. 0. ... 0. 0. 0.]
15178
[0. 0. 0. ... 0. 0. 0.]
15178
[0. 0. 0. ... 0. 0. 0.]
15178
[0. 0. 0. ... 0. 0. 0.]
15178
[0. 0. 0. ... 0. 0. 0.]
15178
[0. 0. 0. ... 0. 0. 0.]
15178
[0. 0. 0. ... 0. 0. 0.]
15178
[0. 1. 0. ... 0. 0. 0.]
15178
[0. 0. 0. ... 0. 0. 0.]
15178
[0. 0. 0. ... 0. 0. 0.]


In [46]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

with tf.device('/device:GPU:0'):
    model = Sequential()
    model.add(Embedding(vocab_size, 10, input_length=max_len-1))
    # y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
    model.add(LSTM(128))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    h = model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200


9258/9258 - 197s - loss: 6.5219 - accuracy: 0.1160


Epoch 2/200


KeyboardInterrupt: 

In [27]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model('nls_model.h5')


In [28]:
def sentence_generation(model, t, current_word, n): 
    init_word = current_word 
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [35]:
print(sentence_generation(model, t, 'hi', 50))

hi containing evoked instrumentality sons evoked evoked evoked evoked evoked evoked evoked evoked evoked evoked evoked evoked evoked evoked cissebr2mesch2 evoked cissebr2mesch2 evoked evoked n8 orallyadministered potential evoked evoked evoked evoked evoked isnt isnt isnt isnt isnt 22thfn evoked fore wong potential potential bean evoked permeability evoked rpgs evoked borrowing evoked


In [ ]:
# --------------------- OLD VERSION ----------------------------- 

In [ ]:
train_text_X = []

for sentences in tokens_stemmed:

    for corpus in sentences:

        train_text_X.append(corpus)


# print(train_text_X)

# Creating NLS

vocab = sorted(list(set(train_text_X)))
vocab.append('UNK')

print('{} unique words'.format(len(vocab)))

word2idx = {u : i for i, u in enumerate(vocab)}
# print(word2idx)

idx2word = np.array(vocab)
# print(idx2word)

text_as_int = np.array([word2idx[c] for c in train_text_X])

print('------------------------------')
print(train_text_X[:20])
print(text_as_int[:20])



In [ ]:
# DATASET
seq_length = 25
examples_per_epoch = len(text_as_int)
sentence_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
# print(examples_per_epoch)
# print(sentence_dataset)
sentence_dataset = sentence_dataset.batch(seq_length+1, drop_remainder=True)
# print(sentence_dataset)
for item in sentence_dataset.take(1):
    print(idx2word[item.numpy()])
    print(item.numpy())


def split_input_target(chunk):
    return [chunk[:-1], chunk[-1]]


# print('--------------------------------------------------------------------')
train_dataset = sentence_dataset.map(split_input_target)
for x, y in train_dataset.take(1):
    print(idx2word[x.numpy()])
    print(x.numpy())
    print(idx2word[y.numpy()])
    print(y.numpy())


In [ ]:
BATCH_SIZE = 128
steps_per_epoch = examples_per_epoch // BATCH_SIZE
BUFFER_SIZE = 10000

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

total_words = len(vocab)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=seq_length),
    tf.keras.layers.LSTM(units=100, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units=100),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def testmodel(epoch, logs):
    
    if epoch % 5 != 0 and epoch != 49:
        return
    # test_sentence = tokens_stemmed[0]
    test_sentence = train_text_X[0]

    next_words = 100

    for _ in range(next_words):
        test_text_X = test_sentence.split(' ')[-seq_length:]
        test_text_X = np.array([word2idx[c] if c in word2idx else word2idx ['UNK'] for c in test_text_X])
        test_text_X = pad_sequences([test_text_X], maxlen=seq_length, padding='pre', value=word2idx['UNK'])
        output_idx = model.predict_classes(test_text_X)
        test_sentence += ' ' + idx2word[output_idx[0]]


    print()
    print(test_sentence)
    print()


testmodelcb = tf.keras.callbacks.LambdaCallback(on_epoch_end=testmodel)
history = model.fit(train_dataset.repeat(), epochs=50, steps_per_epoch=steps_per_epoch, callbacks=[testmodelcb], verbose=2)



In [4]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''

    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장

    sentence = init_word + sentence
    return sentence

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
t = Tokenizer()

In [7]:
print(sentence_generation(model, t, 'entrepreneurshup', 10))


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


UnboundLocalError: local variable 'word' referenced before assignment

In [2]:
#
import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model('nls_model.h5')

# print(model)